<a href="https://colab.research.google.com/github/piyush07dubey/play-store/blob/main/new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [100]:
df=pd.read_csv('googleplaystore_user_reviews.csv', on_bad_lines='skip')

In [101]:
df.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [102]:
df.drop(columns=['Sentiment_Polarity','Sentiment_Subjectivity'], inplace=True)

In [103]:
df.isnull().sum()

,0
App,0
Translated_Review,42912
Sentiment,42905


In [104]:
df['Sentiment'] = df['Sentiment'].fillna(df['Sentiment'].mode()[0])
df['Translated_Review'] = df['Translated_Review'].fillna('')

In [105]:
df.shape

(102054, 3)

In [106]:
!pip install nltk


In [107]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [108]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    # 1. Lowercase
    text = text.lower()

    # 2. Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # 3. Tokenize
    words = word_tokenize(text)

    # 4. Remove stopwords
    words = [word for word in words if word not in stop_words]

    # 5. Lemmatization
    words = [lemmatizer.lemmatize(word) for word in words]

    return " ".join(words)


In [109]:
X = df.drop(columns=['Sentiment'])
y = df['Sentiment']

In [110]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [111]:
X_train['App'] = X_train['App'].apply(clean_text)
X_train['Translated_Review'] = X_train['Translated_Review'].apply(clean_text)
X_test['App'] = X_test['App'].apply(clean_text)
X_test['Translated_Review'] = X_test['Translated_Review'].apply(clean_text)

In [112]:
X_train

,App,Translated_Review
11418,cool reader,really like rotation bug work great using bigg...
86548,free premium vpn finchvpn,nice
42062,cardboard,
28487,etrade mobile,wont let login 2 factor authorization make use...
94601,google slide,
...,...,...
6265,alto adventure,game awesome im quite disappointed alto odysse...
54886,colorful glitter neon butterfly keyboard theme,fake
76820,eat fit diet health free,even work cant get past initial configuration
860,3d live neon weed launcher,


In [113]:
print(type(X_train))   # pandas DataFrame
print(type(y_train))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [115]:
from lightgbm import LGBMClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

# Combine the text features from X_train and X_test into a single series
# The 'clean_text' function should have already been applied to X_train and X_test in a previous cell
X_train_combined = X_train['App'] + ' ' + X_train['Translated_Review']
X_test_combined = X_test['App'] + ' ' + X_test['Translated_Review']

# Initialize and apply TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=5000) # Max_features can be adjusted
X_train_vectorized = vectorizer.fit_transform(X_train_combined)
X_test_vectorized = vectorizer.transform(X_test_combined)

model = LGBMClassifier()
# Fit the model using the vectorized numerical features
model.fit(X_train_vectorized, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 5.675352 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 168272
[LightGBM] [Info] Number of data points in the train set: 81643, number of used features: 3981
[LightGBM] [Info] Start training from score -11.310111
[LightGBM] [Info] Start training from score -11.310111
[LightGBM] [Info] Start training from score -2.068272
[LightGBM] [Info] Start training from score -2.529324
[LightGBM] [Info] Start training from score -0.230850
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

LGBMClassifier()

In [116]:
y_pred = model.predict(X_test_vectorized)


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [117]:
from sklearn.metrics import accuracy_score, classification_report

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.9121062172358042
              precision    recall  f1-score   support

      0.2625       0.00      0.00      0.00         0
    Negative       0.91      0.76      0.83      2604
     Neutral       0.83      0.44      0.57      1593
    Positive       0.92      0.98      0.95     16214

    accuracy                           0.91     20411
   macro avg       0.66      0.54      0.59     20411
weighted avg       0.91      0.91      0.90     20411



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [118]:
print(y.value_counts())


Sentiment
Positive               81027
Negative               12924
Neutral                 8101
0.43333333333333335        1
0.2625                     1
Name: count, dtype: int64


In [132]:
new_text = ["Absolutely horrible service 😡 never buying again."]


In [133]:
cleaned_text = [clean_text(text) for text in new_text]


In [134]:
new_text_vectorized = vectorizer.transform(cleaned_text)


In [135]:
prediction = model.predict(new_text_vectorized)
print("Predicted class:", prediction[0])


Predicted class: Negative


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [136]:
import joblib

# Save model
joblib.dump(model, "lgbm_model.pkl")

# Save vectorizer
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")


['tfidf_vectorizer.pkl']